In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

import datetime

from snobedo.lib.dask_utils import start_cluster, client_ip_and_port
from snobedo.snotel import SnotelLocations, CsvParser

from common import SHARED_STORE, SNOBAL_DIR, SNOTEL_DIR, DATA_DIR, COARSEN_OPTS, HV_PLOT_OPTS

In [ ]:
client = start_cluster(8, 24)
client_ip_and_port(client)

In [ ]:
year = 2022
water_year = f'wy{year}'

## Model with SMRF clouds and solar 

In [ ]:
smrf_eb = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/{water_year}/erw/run*/smrf_energy*.nc',
    parallel=True, chunks={'time': 24},
).sel(time=datetime.time(18))

smrf_albedo = (smrf_eb.albedo_vis * 0.67 + smrf_eb.albedo_ir * 0.33).mean(['x', 'y']).compute()
smrf_albedo_std = (smrf_eb.albedo_vis * 0.67 + smrf_eb.albedo_ir * 0.33).std(['x', 'y']).compute()

## Model run with HRRR clouds and solar + MODIS albedo

In [ ]:
HRRR_modis = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/{water_year}/erw_hrrr_solar_modis_cubic/run*/net_solar.nc',
    parallel=True, chunks={'time': 24},
    drop_variables=['DSWRF', 'illumination_angle'],
).sel(time=datetime.time(18))

HRRR_modis = HRRR_modis.where(HRRR_modis.albedo > 0)

In [ ]:
hm_albedo = HRRR_modis.albedo.mean(['x', 'y']).compute()

In [ ]:
hm_albedo_std = HRRR_modis.albedo.std(['x', 'y']).compute()

In [ ]:
hm_albedo = hm_albedo / 10000
hm_albedo_std = hm_albedo_std / 10000

In [ ]:
# hm_albedo.plot() + hm_albedo_std.plot() + smrf_albedo.plot() + smrf_albedo_std.plot();

## Matplotlib 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
from matplotlib.offsetbox import AnchoredText

In [ ]:
plot_range = pd.date_range(start=f'{year - 1}-10-01', periods=11, freq='MS')

In [ ]:
fig = plt.figure(dpi=300)
ax = fig.gca()

smrf_albedo.plot(ax=ax, lw=1, label='Time-Decay Mean')
ax.fill_between(smrf_albedo.time.values, smrf_albedo + smrf_albedo_std, smrf_albedo - smrf_albedo_std, alpha=0.2)
hm_albedo.plot(ax=ax, lw=1, label='MODIS Mean')
ax.fill_between(hm_albedo.time.values, hm_albedo + hm_albedo_std, hm_albedo - hm_albedo_std, alpha=0.2)

ax.set_xlim(plot_range[0], plot_range[-1])

ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_minor_locator(mdates.MonthLocator(bymonthday=16))
ax.xaxis.set_major_formatter(mticker.NullFormatter())
ax.xaxis.set_minor_formatter(mdates.DateFormatter('%b'))
ax.tick_params(axis='x', which='minor', pad=-0.5)

ax.set_ylim(bottom=0)
ax.set_ylabel('Albedo')
ax.set_xlabel(f'Water Year {year}')
ax.legend(frameon=False);

In [ ]:
client.shutdown()